# Chapitre 3: Outils RAG pour VéloCorp

Ce notebook présente les 6 outils disponibles pour construire un agent RAG multi-sources.

## Setup

In [ ]:
# Clone and install rag_attack
!git clone https://github.com/racousin/rag_attack.git rag_attack_clone
%cd rag_attack_clone
!pip install -e . -q

In [1]:
# Configuration (fill with your credentials)
from rag_attack import set_config

config = {

}

set_config(config)

## Les 6 Outils Disponibles

| Outil | Source | Usage |
|-------|--------|-------|
| `get_crm` | API REST | Opportunités, prospects, commerciaux, analytics |
| `get_erp` | SQL Server | Produits, clients, commandes, factures |
| `get_document_rag` | Azure Search | Documents, FAQs, manuels, emails |
| `get_internet_search` | DuckDuckGo | Recherche web externe |
| `write_file` | Local | Génération de rapports (text, excel) |
| `send_mail` | SMTP | Envoi d'emails |

In [2]:
from rag_attack import get_crm, get_erp, get_document_rag, get_internet_search, write_file, send_mail

# All tools in a list for the agent
tools = [get_crm, get_erp, get_document_rag, get_internet_search, write_file, send_mail]

---
## 1. CRM Tool (`get_crm`)

Accès aux données commerciales via API REST.

In [ ]:
# Voir la signature et description
print(get_crm.name)
print(get_crm.description)
print("\nArgs:", get_crm.args)

In [ ]:
# Exemples d'utilisation

# get_crm.invoke({"query_type": "analytics"})  # KPIs globaux
# get_crm.invoke({"query_type": "opportunities", "status": "Négociation", "min_value": 1000})
# get_crm.invoke({"query_type": "prospects", "min_score": 70})
# get_crm.invoke({"query_type": "sales_reps", "region": "Bretagne"})

---
## 2. ERP Tool (`get_erp`)

Requêtes SQL sur la base de données Azure SQL Server.

In [ ]:
print(get_erp.name)
print(get_erp.description)

In [ ]:
# Exemples d'utilisation (T-SQL syntax!)
# get_erp.invoke({"query": "SELECT TOP 5 name, price FROM products ORDER BY price DESC"})
# get_erp.invoke({"query": "SELECT COUNT(*) as total FROM customers"})
# get_erp.invoke({"query": "SELECT status, COUNT(*) as nb FROM orders GROUP BY status"})

**Schema SQL:**
- `products`: id, model, name, price, stock_quantity, target_market...
- `customers`: id, customer_type, name, email, total_spent, loyalty_level...
- `orders`: id, customer_id, status, total_amount, order_date...
- `order_items`: id, order_id, product_id, quantity, unit_price...
- `support_tickets`: id, customer_id, priority, status, title...
- `invoices`: id, order_id, amount, payment_status...

---
## 3. Document RAG (`get_document_rag`)

Recherche dans Azure Cognitive Search (keyword, vector, hybrid).

In [ ]:
print(get_document_rag.name)
print(get_document_rag.description)

In [3]:
# Exemples d'utilisation
# get_document_rag.invoke({"query": "E-City", "search_type": "keyword"})  # Terme exact
# get_document_rag.invoke({"query": "comment entretenir batterie", "search_type": "vector"})  # Sémantique
# get_document_rag.invoke({"query": "problèmes freins", "search_type": "hybrid"})  # Combiné

"=== Hybrid Search Results ===\n\nResult 1 (score: 0.02):\n  Title: No Title\n  Category: Unknown\n  Content: Manuel Technique - Pro-Sécurité\n        \nBienvenue dans le manuel technique de votre Pro-Sécurité. Ce document contient toutes les informations nécessaires pour l'utilisation, l'entretien et la maintenance de votre vélo.\n\nSPÉCIFICATIONS TECHNIQUES - Pro-Sécurité\n\nPoids: 19.1 kg\nMatériau du cadre: acier\nTailles disponibles: Unique\nCouleurs: noir, blanc, vert\nGarantie: 12 mois\n\nComposants:\n- Freins: Freins à disque hydrauliques\n- Transmission: Dérailleur 21 vitesses\n- Roues: Jantes aluminium doubl...\n\nResult 2 (score: 0.02):\n  Title: No Title\n  Category: Unknown\n  Content: Objet: Problème avec ma commande\n\nBonjour,\n\nJe vous contacte car je rencontre un problème avec mon vélo acheté récemment.\n\nLe problème persiste malgré mes tentatives de résolution.\n\nJ'attends votre retour.\n\nMonique Rivière...\n\nResult 3 (score: 0.02):\n  Title: No Title\n  Categor

**Quand utiliser quel type?**
- `keyword`: Termes exacts, noms de produits, SKUs
- `vector`: Questions conceptuelles, intentions
- `hybrid`: Par défaut quand on hésite

---
## 4. Internet Search (`get_internet_search`)

Recherche web via DuckDuckGo (pas de clé API).

In [ ]:
print(get_internet_search.name)
print(get_internet_search.description)

In [ ]:
# Exemples d'utilisation
# get_internet_search.invoke({"query": "vélo électrique tendances 2024"})
# get_internet_search.invoke({"query": "réglementation vélos France", "num_results": 3})

---
## 5. Writer Tool (`write_file`)

Génération de rapports formatés (text ou Excel).

In [ ]:
print(write_file.name)
print(write_file.description)

In [ ]:
# Exemples d'utilisation
# write_file.invoke({"title": "Analyse Q4", "content": "Ventes +15%...", "report_type": "business"})
# write_file.invoke({"title": "Stock", "content": "Produit||Prix||Stock\nE-City||1299||45", "format": "excel"})

---
## 6. Mail Tool (`send_mail`)

Envoi d'emails via SMTP.

In [ ]:
print(send_mail.name)
print(send_mail.description)

In [ ]:
# Exemples d'utilisation
# your_user_email=""
# send_mail.invoke({"to": your_user_email, "subject": "Commande", "body": "Votre commande..."})
# send_mail.invoke({"to": your_user_email, "subject": "Rapport", "body": "<h1>Rapport</h1>", "html": True})

---
## Récapitulatif: Comment l'Agent Choisit

```
Question Utilisateur → LLM lit les descriptions → Choisit l'outil → Exécute → Répond
```

**La qualité de la description détermine si le LLM utilise le bon outil avec les bons paramètres.**